# Convert csv to DwC

Workflow
1. Convert column headers to DwC headers
2. Align values to DwC standards, e.g., date time to ISO 8601
3. Manage non-standard fields, when present. These are 'dynamic properties,' e.g., length, weight, temperature, etc. 

## To Do
See the taxonomy and dynamic properties comments at the bottom of this notebook.

1. ~~add taxon fields~~
2. ~~split `Type` into `lifeStage` and `sex`~~
3. ~~confirm that timezone is UTC~~ (Its not, in local pst)

Resources:

http://rs.tdwg.org/dwc/terms/index.htm

http://www.iobis.org/manual/darwincore/

In [6]:
import pandas as pd
import urllib.request, json

In [7]:
ls

ACCESS euphausiid DwC conversion.ipynb*
ACCESS_euphausiids_Tucker_2004-14_Hoop_2004-13_corrected_depths.csv*


In [18]:
# import CSV
df = pd.read_csv("ACCESS_euphausiids_Tucker_2004-14_Hoop_2004-13_corrected_depths.csv")
df.head()

,Date,ID_Jar,Net,Lat_IN,Lon_IN,Depth_Min,Depth_Max,Volume_m3,Time_IN,Species_Name,Type,Length_l,Length_h,Tot_num
0,7/26/04,1,Hoop,38.046100,-123.563000,-1.0,-50.0,355.79,9:41:00,Euphausia pacifica,imm f,14.0,14.0,1.0
1,7/26/04,1,Hoop,38.046100,-123.563000,-1.0,-50.0,355.79,9:41:00,Thysanoessa spinifera,juv,7.0,7.2,2.0
2,7/26/04,1,Hoop,38.046100,-123.563000,-1.0,-50.0,355.79,9:41:00,Euphausia pacifica,zoea,3.5,4.0,4.0
3,7/26/04,2,Hoop,38.048781,-123.467652,-1.0,-50.0,367.10,11:18:00,Euphausia pacifica,f,12.0,12.0,1.0
4,7/26/04,2,Hoop,38.048781,-123.467652,-1.0,-50.0,367.10,11:18:00,Euphausia pacifica,juv,5.0,7.0,136.0


In [19]:
# convert column headers to DwC
df = df.rename(columns = {'Lat_IN': 'decimalLatitude',
                          'Lon_IN': 'decimalLongitude',
                          'Depth_Min': 'minimumDepthInMeters',
                          'Depth_Max': 'maximumDepthInMeters',
                          'Volume_m3': 'sampleSizeValue',
                          'Tot_num': 'individualCount',
                          'Net': 'samplingProtocol',
                          'ID_Jar': 'eventID',
                          'Length_l': 'lengthMinimumInMeters', #not DwC
                          'Length_h': 'lengthMaximumInMeters'}) #not DwC

# convert mm to m
df['lengthMinimumInMeters'] = df['lengthMinimumInMeters'] / 1000
df['lengthMaximumInMeters'] = df['lengthMaximumInMeters'] / 1000

# add required DwC columns
df['sampleSizeUnit'] = 'm3'
df['occurrenceStatus'] = 'present'
df['basisOfRecord'] = 'HumanObservation'

# Added Identification reference from data description
df['identificationReferences'] = 'Brinton, E, Ohman MD, Townsend AW, Knight MD, Bridgeman A.  1999.  Euphausiids of the world ocean. ( Expert-center for Taxonomic Identification , Scripps Institution of Oceanography , Eds.)., Amsterdam: University of Amsterdam, ETI'
# Changes minimum depth to positive values, should be depth below the surface according to:
# https://terms.tdwg.org/wiki/dwc:minimumDepthInMeters
df['minimumDepthInMeters'] = df['minimumDepthInMeters'] * -1
df['maximumDepthInMeters'] = df['maximumDepthInMeters'] * -1

# modify Net column
df['samplingProtocol'] = df['samplingProtocol'] + ' Net'

# create an occurrence id column by concat ID_Jar with integer for each unique species in jar
df = df.sort_values(by = ['eventID']).reset_index(drop=True)
df['occurrenceID'] = df['eventID'].map(str) + '_' + (df.groupby('eventID').cumcount()+1).map(str)

df.head()

,Date,eventID,samplingProtocol,decimalLatitude,decimalLongitude,minimumDepthInMeters,maximumDepthInMeters,sampleSizeValue,Time_IN,Species_Name,Type,lengthMinimumInMeters,lengthMaximumInMeters,individualCount,sampleSizeUnit,occurrenceStatus,basisOfRecord,identificationReferences,occurrenceID
0,7/26/04,1,Hoop Net,38.046100,-123.563000,1.0,50.0,355.79,9:41:00,Euphausia pacifica,imm f,0.0140,0.0140,1.0,m3,present,HumanObservation,"Brinton, E, Ohman MD, Townsend AW, Knight MD, ...",1_1
1,7/26/04,1,Hoop Net,38.046100,-123.563000,1.0,50.0,355.79,9:41:00,Thysanoessa spinifera,juv,0.0070,0.0072,2.0,m3,present,HumanObservation,"Brinton, E, Ohman MD, Townsend AW, Knight MD, ...",1_2
2,7/26/04,1,Hoop Net,38.046100,-123.563000,1.0,50.0,355.79,9:41:00,Euphausia pacifica,zoea,0.0035,0.0040,4.0,m3,present,HumanObservation,"Brinton, E, Ohman MD, Townsend AW, Knight MD, ...",1_3
3,7/26/04,2,Hoop Net,38.048781,-123.467652,1.0,50.0,367.10,11:18:00,Euphausia pacifica,f,0.0120,0.0120,1.0,m3,present,HumanObservation,"Brinton, E, Ohman MD, Townsend AW, Knight MD, ...",2_1
4,7/26/04,2,Hoop Net,38.048781,-123.467652,1.0,50.0,367.10,11:18:00,Euphausia pacifica,juv,0.0050,0.0070,136.0,m3,present,HumanObservation,"Brinton, E, Ohman MD, Townsend AW, Knight MD, ...",2_2


In [20]:
# combine date and time into ISO 8601 "eventDate"
df['eventDate'] = pd.to_datetime(df['Date'] + ' ' + df['Time_IN'])
df = df.drop(columns = ['Date', 'Time_IN'])
# I wasn't sure of the best way to handle this, but this is what I've done for other projects,
# Seems like overkill vs just adding timedelta
df.index = df['eventDate']
df = df.drop(columns=['eventDate']) # this drops the 'extra' eventDate column, but keeps the eventDate index
# Safer way of handeling timezone conversion
df = df.tz_localize('US/Pacific')
df = df.tz_convert('UTC')

df.head()

,eventID,samplingProtocol,decimalLatitude,decimalLongitude,minimumDepthInMeters,maximumDepthInMeters,sampleSizeValue,Species_Name,Type,lengthMinimumInMeters,lengthMaximumInMeters,individualCount,sampleSizeUnit,occurrenceStatus,basisOfRecord,identificationReferences,occurrenceID
eventDate,,,,,,,,,,,,,,,,,
2004-07-26 16:41:00+00:00,1,Hoop Net,38.046100,-123.563000,1.0,50.0,355.79,Euphausia pacifica,imm f,0.0140,0.0140,1.0,m3,present,HumanObservation,"Brinton, E, Ohman MD, Townsend AW, Knight MD, ...",1_1
2004-07-26 16:41:00+00:00,1,Hoop Net,38.046100,-123.563000,1.0,50.0,355.79,Thysanoessa spinifera,juv,0.0070,0.0072,2.0,m3,present,HumanObservation,"Brinton, E, Ohman MD, Townsend AW, Knight MD, ...",1_2
2004-07-26 16:41:00+00:00,1,Hoop Net,38.046100,-123.563000,1.0,50.0,355.79,Euphausia pacifica,zoea,0.0035,0.0040,4.0,m3,present,HumanObservation,"Brinton, E, Ohman MD, Townsend AW, Knight MD, ...",1_3
2004-07-26 18:18:00+00:00,2,Hoop Net,38.048781,-123.467652,1.0,50.0,367.10,Euphausia pacifica,f,0.0120,0.0120,1.0,m3,present,HumanObservation,"Brinton, E, Ohman MD, Townsend AW, Knight MD, ...",2_1
2004-07-26 18:18:00+00:00,2,Hoop Net,38.048781,-123.467652,1.0,50.0,367.10,Euphausia pacifica,juv,0.0050,0.0070,136.0,m3,present,HumanObservation,"Brinton, E, Ohman MD, Townsend AW, Knight MD, ...",2_2


## taxonomy

Assuming marine taxa (examples for Euphausiidae):

1. provide WoRMS AphiaID for all species as `taxonID`, e.g., 110671
2. `Species_Name` --> `scientificName`, e.g., Euphausiidae
3. WoRMS LSID --> `scientificNameID`, e.g., urn:lsid:marinespecies.org:taxname:110671
4. `nameAccordingToID` e.g., WORMS

WoRMS matching is outlined here: http://www.iobis.org/manual/namematching/

In [21]:
# Generate a list of unique scientific names
df.groupby('Species_Name').count()
distinct_species = df['Species_Name'].unique()
species_df = pd.DataFrame(data = distinct_species, columns = ['sci_name'])
species_df

,sci_name
0,Euphausia pacifica
1,Thysanoessa spinifera
2,Nematoscelis difficilis
3,Euphausiidae
4,Thysanoessa inspinata
5,Thysanoessa gregaria
6,Euphausia recurva
7,Stylocheiron spp
8,Euphausia spp
9,Stylocheiron longicorne


In [22]:
species_df.to_csv('species.csv') # Can be used for manual input

## dynamic properties

#### type (life stage and sex)
split type into `lifeStage` and `sex` columns


In [23]:
def get_worms_from_scientific_name(sci_name):
    '''
    Using WORMS REST Api, retrieve the id given a scientific name. This method is sensitive to syntax errors.
    
    Returns:
        - scientificName: Worms specified scientific name
        - scientificNameID: Worms specific if for scientific name
    '''
    sci_name_url = sci_name.replace(' ','%20') #replace space with utf-8 for url building
    _url = 'http://www.marinespecies.org/rest/AphiaRecordsByNames?scientificnames%5B%5D='+ sci_name_url + '&like=false&marine_only=true'
    try:
        with urllib.request.urlopen(_url) as url:
            data = json.loads(url.read().decode())
            return (data[0][0]['scientificname'], data[0][0]['lsid'], data[0][0]['AphiaID'])
    except Exception as e:
        if len(sci_name_url.split('%20')) > 1: #If species is unknown and listed as spp. or sp.
            return get_worms_from_scientific_name(sci_name_url.split('%20')[0])
        else:
            print("Url didn't work, check the Scientific name, ", sci_name)

In [24]:
# Build Name and Id dictionary
name_id_dic = {}
name_dic = {}
id_dic = {}

for sci_name in species_df['sci_name']:
    sci_name = sci_name.strip()
    sname, sname_id, id = get_worms_from_scientific_name(sci_name)
    name_id_dic[sci_name] = sname_id
    name_dic[sci_name] = sname
    id_dic[sci_name] = id
print(id_dic)

{'Euphausia pacifica': 237851, 'Thysanoessa spinifera': 237874, 'Nematoscelis difficilis': 237861, 'Euphausiidae': 110671, 'Thysanoessa inspinata': 237872, 'Thysanoessa gregaria': 110707, 'Euphausia recurva': 221047, 'Stylocheiron spp': 110678, 'Euphausia spp': 110673, 'Stylocheiron longicorne': 110703, 'Nyctiphanes simplex': 237864, 'Stylocheiron elongatum': 110702, 'Tessarabrachion oculatum': 237870, 'Nematobrachion flexipes': 110692, 'Stylocheiron abbreviatum': 110699, 'Stylocheiron affine': 110700}


In [25]:
# Add new column to dataframe for WORMS id

df['scientificNameID'] = df['Species_Name'].str.strip()
df.replace({'scientificNameID': name_id_dic}, inplace=True)

df['taxonID'] = df['Species_Name'].str.strip()
df.replace({'taxonID': id_dic}, inplace=True)

df['scientificName'] = df['Species_Name'].str.strip()
df['scientificName'] = df.scientificName.map(name_dic)

df['nameAccordingToID'] = 'WoRMS'

df = df.drop(columns=['Species_Name'])

In [26]:
# Break out type into sex and lifeStage
types_raw = list(df['Type'].unique())
def get_life_stage(type_name):
    ''' Return a life stage value given these conditions'''
    type_name = str(type_name).strip()
    if 'imm' in type_name:
        return('immature')
    elif 'adult' in type_name:
        return('adult')
    elif 'juv' in type_name:
        return('juvenile')
    elif 'zoea' in type_name:
        return('zoea')
    elif 'nauplii' in type_name:
        return('nauplii')
    elif (type_name == 'm' or type_name == 'f'):
        return('adult') # This is specified in the data description
    else:
        return('unknown') # Question about this?
    
df['lifeStage'] = df['Type'].apply(get_life_stage)

In [27]:
# Breakout the sex from the type
def get_sex(type_name):
    ''' Return the sex if availible from the type catagory'''
    t = str(type_name).strip()
    if 'f' in t:
        return('female')
    elif 'm' == t:
        return('male')
    elif t.split(' ')[-1] == 'm':
        return('male')
    else:
        return('indeterminate')

df['sex'] = df['Type'].apply(get_sex)
df = df.drop(columns = ['Type'])

In [28]:
# Save the DwC formatted csv
df.to_csv('access_trawl_formatted.csv')